# pinnFoam - Grid Search for hyperparameter tuning

In [ ]:
import numpy as np
import itertools
import os
import subprocess


# Define the parameter space lists
layers = [4,5,6,7]
nodes_per_layer = [10,20,30]
optimizer_step = [1e-04,1e-03]
max_iterations = [3000]
epsilon = [1e-05]

# Prepare the list of parameter space lists
parameter_space = [
    layers,
    nodes_per_layer,
    optimizer_step,
    max_iterations, 
    epsilon
]

# Select the cases that will be used as OpenFOAM input 
# - Useful when OpenFOAM parameters are aditionally varied
cases = ["unit_box_domain"] 
max_parallel_processes = 3 # Maximal number of parallel jobs 

# For all OpenFOAM cases
for case_name in cases:
    active_processes = []
    # Loop over the cartesian product of parameter vectors
    for i,param_vector in enumerate(itertools.product(*parameter_space)):
        print ("Run id = %d , running parameter vector %s" % (i, param_vector))
        # Construct the hiddenLayers option for pinnFoam
        hidden_layers = ("(")
        for layers in range(param_vector[0]):
            hidden_layers = hidden_layers + str(param_vector[1]) + " "
        hidden_layers = hidden_layers + ")"
        # Start the training
        active_processes.append(subprocess.Popen(['pinnFoam',
            '-case', case_name,
            '-hiddenLayers', hidden_layers, 
            '-optimizerStep', str(param_vector[2]), 
            '-maxIterations', str(param_vector[3]), 
            '-epsilon', str(param_vector[4])], stdout=subprocess.DEVNULL))
        # If max number of parallel processes is reached
        if (i % max_parallel_processes == 0):
            # Wait for active processes to finish
            for process in active_processes:
                process.wait()
            active_processes.clear()

Run id = 0 , running parameter vector (4, 10, 0.0001, 3000, 1e-05)
Run id = 1 , running parameter vector (4, 10, 0.001, 3000, 1e-05)
Run id = 2 , running parameter vector (4, 20, 0.0001, 3000, 1e-05)
Run id = 3 , running parameter vector (4, 20, 0.001, 3000, 1e-05)
Run id = 4 , running parameter vector (4, 30, 0.0001, 3000, 1e-05)
Run id = 5 , running parameter vector (4, 30, 0.001, 3000, 1e-05)
Run id = 6 , running parameter vector (5, 10, 0.0001, 3000, 1e-05)
